In [1]:
# Dependencies
import numpy as np
import pandas as pd
from datetime import datetime
import time
import glob
import quandl
import os

In [2]:
# define a function to read and handle bitcoin price data frame
def generate_BTC_Price_df():
    
    # read in bitcoin price file
    coin = pd.read_csv("bitcoinPrice.csv", index_col = 0)
    
    # convert date info from string to datetime.date object
    Converted_date = []
    for i in range(len(coin)):
        date = datetime.date(datetime.strptime(coin["Date"][i], '%Y-%m-%d'))
        Converted_date.append(date)
    coin["Converted_Date"] = Converted_date
    
    # adjust the data frame: leave two columns - Converted_date & BTC_Price
    coin.drop(labels="Date", axis=1, inplace=True)
    coin = coin[['Converted_Date', 'Close']]
    coin = coin.rename(columns={'Close': 'BTC_Price'})
    
    return coin

In [3]:
# define a function to read in single CSV file, clean, groupby
def single_CSV_clean(filename):
    
    # read in csv file
    df = pd.read_csv(filename, index_col = 0)
    
    # clean up data frame
    mask = df['Tweet Date'].apply(lambda x: len(str(x)) == 30)
    mask1 = df['Language'] == 'en'
    mask2 = df['Compound'].notnull()
    df = df[mask&mask1&mask2]
    
    # convert Tweet Date to datetime.date object and add to new column
    timelist = []
    for tweet_date in df['Tweet Date']: 
        tweet_datetime = datetime.date(pd.to_datetime(tweet_date))
        timelist.append(tweet_datetime)
    df['Converted_Date'] = timelist
    
    # generate new data frame after groupby & calculation
    cleaned_df = df.groupby('Converted_Date').agg({"Text": "count", "Compound" : "mean"})
    cleaned_df.reset_index(inplace=True)
    
    # split to Tweets data frame and Compound data frame, prepare for merge
    Tweet_df = cleaned_df.drop(labels="Compound", axis=1)
    Tweet_df.columns = [["Converted_Date", df["Account Name"][0]]]
    compound_df = cleaned_df.drop(labels="Text", axis=1)
    compound_df.columns = [["Converted_Date", df["Account Name"][0]]]
    
    return Tweet_df, compound_df

In [4]:
# define a function to handle multiple CSV files in a same folder, and finish merge
def generate_merged_tables(folder, coin_df):
    
    # define file path
    csvpath = os.path.join(folder, '*.csv')
    
    # generate CSV file list
    file_list = glob.glob(csvpath)
    
    # initiate merged data frame using bitcoin data frame
    merged_tweet_df = coin_df
    merged_compound_df = coin_df
    
    # loop through csv files, extract info and merge with bitcoin info
    for file in file_list:
        Tweet_df, compound_df = single_CSV_clean(file)
        merged_tweet_df = pd.merge(merged_tweet_df, Tweet_df, on="Converted_Date", how="outer")
        merged_compound_df = pd.merge(merged_compound_df, compound_df, on="Converted_Date", how="outer")
    
    # save two merged files
    merged_tweet_df.to_csv("merged_tweet_file.csv")
    merged_compound_df.to_csv("merged_compound_file.csv")

In [6]:
coin_df = generate_BTC_Price_df()
generate_merged_tables("CSVfiles-individual", coin_df)